# Logistic Regression

### Setup

In [1]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np
import random

from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from scipy.special import expit

In [2]:
#setting a random seed for reproducibility
random.seed(4)

In [3]:
#getting the data
with open('realAccountData.json', 'r') as f:
            realData = json.load(f)
with open('fakeAccountData.json', 'r') as f:
            fakeData = json.load(f)
allData = realData + fakeData #note that the data is ordered right now
random.shuffle(allData) #so this shuffles the list just in case

In [4]:
# allData[1] #making sure the combination works

In [5]:
#making sure the data came in correctly
# fakeData[0], realData[0]
# realData[1]

In [6]:
#turning into numpy arrays
numpy_real = np.array(realData)
# print(numpy_real[1]) #making sure things work
numpy_fake = np.array(fakeData)
# print(numpy_fake[1]) #making sure things work
numpy_all = np.array(allData)
# print(numpy_all[1]) #making sure things work


## Linear Logistic Regression

In [7]:
#turning into pandas dataframes
allData_df = pd.DataFrame(allData)

#adding the ones and seperating the data
allData_df.insert(0, 'Ones', 1)
cols = allData_df.shape[1]
X = allData_df.iloc[:,0:cols-1] # iloc slicing function
Y = allData_df.iloc[:,cols-1:cols]


display(allData_df) #making sure this works
# display(X) #making sure this works
# display(^) #making sure this works

,Ones,userFollowerCount,userFollowingCount,userBiographyLength,userMediaCount,userHasProfilPic,userIsPrivate,usernameDigitCount,usernameLength,isFake
0,1,65,67,79,116,1,0,0,8,0
1,1,219,337,0,0,1,1,1,7,0
2,1,252,509,0,28,1,1,0,11,0
3,1,58,2132,0,0,0,0,0,14,1
4,1,452,481,0,82,1,1,0,6,0
...,...,...,...,...,...,...,...,...,...,...
1189,1,234,658,93,2,1,0,0,14,0
1190,1,846,743,22,38,1,0,0,12,0
1191,1,384,219,51,11,1,1,0,12,0
1192,1,323,405,0,69,1,1,0,10,0


In [40]:
#train test split being made
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# and normalizing the X data
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

x_train, x_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.25, random_state=4)
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=4)

### NEED TO DO: PDA and make a 2d plot

In [41]:
#note: tried to do logistic regression manually, but it was not really working. 
#i was having float overflows and divide by 0 errors

#note: the unscaled sklearn algorithms take a while to run (a few minutes) especially the unscalpolynomial algorithm

In [42]:
# Sklearn linear logistic regression version 
y_train_array = np.ravel(y_train)
# print(Y_array)
linear_log = LogisticRegression(random_state=4, max_iter=10000).fit(x_train, y_train_array)

linear_log.score(x_test, y_test)
#result is .9431
#unscaled the result is .9598

0.9431438127090301

In [44]:
# Sklearn polynomial logistic regression version 
poly = PolynomialFeatures(degree=2)
x_train_mod = x_train[:,1:cols] #removing the ones column from X
X_poly = poly.fit_transform(x_train_mod)

poly_log = LogisticRegression(random_state=4, max_iter=10000).fit(X_poly, y_train_array)

x_test_mod = x_test[:,1:cols] #removing the ones column from X
X_poly_test = poly.fit_transform(x_test_mod)
poly_log.score(X_poly_test,y_test)
#result is .9565 
#it is much faster when scaled (only a few seconds)

0.9565217391304348

In [ ]:
#figure out how to add a regularization term

In [35]:
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=4)
# poly = PolynomialFeatures(degree=2)
# x_train_mod = x_train.iloc[:,1:cols] #removing the ones column from X
# X_poly = poly.fit_transform(x_train_mod)

# poly_log = LogisticRegression(random_state=4, max_iter=10000).fit(X_poly, y_train_array)

# x_test_mod = x_test.iloc[:,1:cols] #removing the ones column from X
# X_poly_test = poly.fit_transform(x_test_mod)
# poly_log.score(X_poly_test,y_test)
# #took 8 minutes, maxed out iterations and did not converge
# #result is .9766